In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

## Carga de imagenes

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving archive.zip to archive.zip


In [ ]:
!mkdir buterfly

In [ ]:
import zipfile
import os

# Especifica la ruta del archivo zip
zip_file_path = 'archive.zip'  # Cambia esto por la ruta de tu archivo zip
output_dir = './buterfly'  # Cambia esto por la ruta donde quieres descomprimir el archivo

# Crea el directorio de salida si no existe
os.makedirs(output_dir, exist_ok=True)

# Descomprimir el archivo zip
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print("Descompresión completa.")

Descompresión completa.


## Normalizacion de imagenes

- Paso 1: cargue el conjunto de datos de imágenes: usando la función `torchvision.datasets.ImageFolder`, que carga automáticamente todas las imágenes en una carpeta y las asigna a sus respectivas clases según los nombres de sus carpetas.
- Paso 2: Calcule la media y la desviación estándar del conjunto de datos:  usando la función `torch.utils.data.DataLoader` y una función personalizada para calcular la media y la desviación estándar del conjunto de datos.
- Paso 3: Normalizar el conjunto de datos: usando la función `transforms.Normalize`.


Paso 0: Definimos la funcion de transformacion

In [ ]:
# Define normalization parameters (mean and std) for pre-trained models
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Define transforms for training, validation, and testing
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'valid': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

Paso 1: cargue el conjunto de datos de imágenes:

In [ ]:
# Define the directories
data_dir = 'buterfly'  # Replace with your dataset path
train_dir = f'{data_dir}/train'
valid_dir = f'{data_dir}/valid'
test_dir = f'{data_dir}/test'

In [ ]:
# Load the datasets
image_datasets = {
    'train': datasets.ImageFolder(train_dir, transform=data_transforms['train']),
    'valid': datasets.ImageFolder(valid_dir, transform=data_transforms['valid']),
    'test': datasets.ImageFolder(test_dir, transform=data_transforms['test']),
}

In [ ]:
# Create data loaders
batch_size = 32
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True),
    'valid': DataLoader(image_datasets['valid'], batch_size=batch_size, shuffle=False),
    'test': DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=False),
}

In [ ]:
# Get class names
class_names = image_datasets['train'].classes
num_classes = len(class_names)


Definimos un modelo de CNN

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            # Convolutional Layer 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            # Convolutional Layer 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            # Convolutional Layer 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128 * 28 * 28, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        # Flatten the tensor
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Initialize the model
model = SimpleCNN(num_classes=num_classes)

Definimos Loss Function and Optimizer

In [ ]:
# Define device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Entrenamos el Modelo

In [ ]:
num_epochs = 20  # Set the number of epochs

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'valid']:
        if phase == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        running_loss = 0.0
        running_corrects = 0

        # Iterate over data
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Backward pass and optimize in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        # Calculate epoch loss and accuracy
        epoch_loss = running_loss / len(image_datasets[phase])
        epoch_acc = running_corrects.double() / len(image_datasets[phase])

        print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

print('Training complete')


Epoch 1/20
----------
Train Loss: 3.9903 Acc: 0.0979
Valid Loss: 2.8260 Acc: 0.3240
Epoch 2/20
----------
Train Loss: 2.7094 Acc: 0.3199
Valid Loss: 1.7412 Acc: 0.5740
Epoch 3/20
----------


Evaluamos el Modelo

In [ ]:
model.eval()  # Set model to evaluate mode
running_corrects = 0

with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        running_corrects += torch.sum(preds == labels.data)

test_acc = running_corrects.double() / len(image_datasets['test'])
print(f'Test Accuracy: {test_acc:.4f}')

Guardamos el modelo

In [ ]:
torch.save(model.state_dict(), 'butterfly_classifier.pth')